# Strava API Pipeline Workbook

## Basic Setup

In [1]:
import requests
import json
import datetime
from datetime import datetime
from datetime import date
import time
import pandas as pd
import os
from dotenv import load_dotenv
import sqlite3

pd.set_option('display.max_columns', None)

# Developer Docs
# https://developers.strava.com/docs/getting-started/#basic

# Streams API Details
# https://developers.strava.com/docs/reference/#api-Streams-getActivityStreams

# Replace with your actual credentials
load_dotenv(dotenv_path="secrets.env")
CLIENT_ID = os.environ.get("CLIENT_ID")
CLIENT_SECRET = os.environ.get("CLIENT_SECRET")
REFRESH_TOKEN = os.environ.get("REFRESH_TOKEN")

DB_PATH = 'strava_data.db'

### Error Troubleshooting

HTTP error occurred: 401 Client Error: Unauthorized for url: https://www.strava.com/api/v3/athlete/activities?page=1&per_page=1

Go to this link, and note the scopes being used at the end of the URL - 
https://www.strava.com/oauth/authorize?client_id=64543&response_type=code&redirect_uri=http://localhost/exchange_token&approval_prompt=force&scope=profile:read_all,activity:read_all

More details on scope here - https://developers.strava.com/docs/authentication/#detailsaboutrequestingaccess

Extract the auth code from the reply URL and assign it using the below cell.

Then, run the contents of the Code to Exchange Auth Code for Auth Token section to get a valid AUTH_TOKEN

In [82]:
AUTH_CODE = "b9ac405e2fa9214157b128942becabf9326b4c74"

## Refresh Auth Token
### STill Figuring this one out
#### The Auth Code input here should be the code extracted from the blank webpage after a user approves a scope request

In [27]:
# old gemini code don't know if useful or not
# This SHOULD work 
def refresh_access_token(client_id, client_secret, refresh_token):
    """Refreshes the access token using the refresh token."""
    response = requests.post(
        url="https://www.strava.com/oauth/token",
        data={
            "client_id": client_id,
            "client_secret": client_secret,
            "grant_type": "refresh_token",
            "refresh_token": refresh_token,
        },
    )
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
    return response.json()

In [28]:
try:
    AUTH_TOKEN = refresh_access_token(CLIENT_ID, CLIENT_SECRET, REFRESH_TOKEN)["access_token"]

except requests.exceptions.HTTPError as e:
    print(f"HTTP error occurred: {e}")
except json.JSONDecodeError as e:
    print(f"JSON decode error occurred: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
    
print(AUTH_TOKEN)

84397cd47707a59cd9c93c559cc8900297a2edcf


## Get Auth Token
#### The Auth Code input here should be the code extracted from the blank webpage after a user approves a scope request

In [37]:
def get_auth_token(client_id, client_secret, auth_code):
    """Refreshes the access token using the refresh token."""
    response = requests.post(
        url="https://www.strava.com/oauth/token",
        data={
            "client_id": client_id,
            "client_secret": client_secret,
            "grant_type": "authorization_code",
            "code": auth_code,
        },
    )
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
    return response.json()

#### Code to Exchange Auth Code for Auth Token

In [83]:
try:
    AUTH_TOKEN = get_auth_token(CLIENT_ID, CLIENT_SECRET, AUTH_CODE)["access_token"]

except requests.exceptions.HTTPError as e:
    print(f"HTTP error occurred: {e}")
except json.JSONDecodeError as e:
    print(f"JSON decode error occurred: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
    
print(AUTH_TOKEN)

acc5596ec3eb974ce0036c9fdc228ecee00fbf12


## Base Functions for retreiving activities and timeseries data from Strava API

In [86]:
def get_activities(access_token, page=1, per_page=30, **optional_parameters):
    """Retrieves activities from the Strava API.
    Optional parameters should be provided at the end of the call like so:
    before = epoch_timestamp, after = epoch_timestamp
    """
    url = f"https://www.strava.com/api/v3/athlete/activities"
    headers = {"Authorization": f"Bearer {access_token}"}
    params = {"page": page, "per_page": per_page}
    params.update(optional_parameters)
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    return response.json()

def get_streams(access_token, keys, activity_id):
    """Retrieves activities from the Strava API."""
    url = f"https://www.strava.com/api/v3/activities/" + str(activity_id) + "/streams"
    headers = {"Authorization": f"Bearer {access_token}"}
    params = {"keys" : keys, "key_by_type": True}
    # valid keys includes ["time", "distance", "latlng", "altitude", "heartrate", "cadence", "watts"]
    # https://developers.strava.com/docs/reference/#api-models-StreamSet
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    return response.json()

def get_gear(access_token, gear_id):
    """Retrieves activities from the Strava API."""
    url = f"https://www.strava.com/api/v3/gear/" + str(gear_id)
    headers = {"Authorization": f"Bearer {access_token}"}
    # params = {"id" : gear_id}
    # valid keys includes ["time", "distance", "latlng", "altitude", "heartrate", "cadence", "watts"]
    # https://developers.strava.com/docs/reference/#api-models-StreamSet
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

## Base functions for storing data in db

#### Initialize Activities Table

In [6]:
def initialize_database():
    """Create the SQLite database and full 'activities' table."""
    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()

    c.execute('''
    CREATE TABLE IF NOT EXISTS activities (
        id INTEGER PRIMARY KEY,
        resource_state INTEGER,
        athlete_id INTEGER,
        athlete_resource_state INTEGER,
        name TEXT,
        distance REAL,
        moving_time INTEGER,
        elapsed_time INTEGER,
        total_elevation_gain REAL,
        type TEXT,
        sport_type TEXT,
        workout_type INTEGER,
        start_date TEXT,
        start_date_local TEXT,
        timezone TEXT,
        utc_offset REAL,
        location_city TEXT,
        location_state TEXT,
        location_country TEXT,
        achievement_count INTEGER,
        kudos_count INTEGER,
        comment_count INTEGER,
        athlete_count INTEGER,
        photo_count INTEGER,
        map_id TEXT,
        map_summary_polyline TEXT,
        map_resource_state INTEGER,
        trainer BOOLEAN,
        commute BOOLEAN,
        manual BOOLEAN,
        private BOOLEAN,
        visibility TEXT,
        flagged BOOLEAN,
        gear_id TEXT,
        start_latlng TEXT,
        end_latlng TEXT,
        average_speed REAL,
        max_speed REAL,
        average_cadence REAL,
        average_watts REAL,
        max_watts INTEGER,
        weighted_average_watts INTEGER,
        device_watts BOOLEAN,
        kilojoules REAL,
        has_heartrate BOOLEAN,
        average_heartrate REAL,
        max_heartrate REAL,
        heartrate_opt_out BOOLEAN,
        display_hide_heartrate_option BOOLEAN,
        elev_high REAL,
        elev_low REAL,
        upload_id INTEGER,
        upload_id_str TEXT,
        external_id TEXT,
        from_accepted_tag BOOLEAN,
        pr_count INTEGER,
        total_photo_count INTEGER,
        has_kudoed BOOLEAN,
        import_date TEXT
    )
    ''')

    conn.commit()
    conn.close()

#### Activities Individual DB Entry Write

In [7]:
import json

def insert_activities(activity_list):
    """Insert activity records, skipping those with duplicate 'id'."""
    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()

    for activity in activity_list:
        try:
            c.execute('''
            INSERT INTO activities VALUES (
                :id, :resource_state, 
                :athlete_id, :athlete_resource_state,
                :name, :distance, :moving_time, :elapsed_time, :total_elevation_gain,
                :type, :sport_type, :workout_type, :start_date, :start_date_local,
                :timezone, :utc_offset, :location_city, :location_state, :location_country,
                :achievement_count, :kudos_count, :comment_count, :athlete_count, :photo_count,
                :map_id, :map_summary_polyline, :map_resource_state,
                :trainer, :commute, :manual, :private, :visibility, :flagged, :gear_id,
                :start_latlng, :end_latlng,
                :average_speed, :max_speed, :average_cadence, :average_watts,
                :max_watts, :weighted_average_watts, :device_watts, :kilojoules,
                :has_heartrate, :average_heartrate, :max_heartrate,
                :heartrate_opt_out, :display_hide_heartrate_option,
                :elev_high, :elev_low,
                :upload_id, :upload_id_str, :external_id, :from_accepted_tag,
                :pr_count, :total_photo_count, :has_kudoed, :import_date
            )
            ''', {
                "id": activity["id"],
                "resource_state": activity.get("resource_state"),
                "athlete_id": activity.get("athlete", {}).get("id"),
                "athlete_resource_state": activity.get("athlete", {}).get("resource_state"),
                "name": activity.get("name"),
                "distance": activity.get("distance"),
                "moving_time": activity.get("moving_time"),
                "elapsed_time": activity.get("elapsed_time"),
                "total_elevation_gain": activity.get("total_elevation_gain"),
                "type": activity.get("type"),
                "sport_type": activity.get("sport_type"),
                "workout_type": activity.get("workout_type"),
                "start_date": activity.get("start_date"),
                "start_date_local": activity.get("start_date_local"),
                "timezone": activity.get("timezone"),
                "utc_offset": activity.get("utc_offset"),
                "location_city": activity.get("location_city"),
                "location_state": activity.get("location_state"),
                "location_country": activity.get("location_country"),
                "achievement_count": activity.get("achievement_count"),
                "kudos_count": activity.get("kudos_count"),
                "comment_count": activity.get("comment_count"),
                "athlete_count": activity.get("athlete_count"),
                "photo_count": activity.get("photo_count"),
                "map_id": activity.get("map", {}).get("id"),
                "map_summary_polyline": activity.get("map", {}).get("summary_polyline"),
                "map_resource_state": activity.get("map", {}).get("resource_state"),
                "trainer": activity.get("trainer"),
                "commute": activity.get("commute"),
                "manual": activity.get("manual"),
                "private": activity.get("private"),
                "visibility": activity.get("visibility"),
                "flagged": activity.get("flagged"),
                "gear_id": activity.get("gear_id"),
                "start_latlng": json.dumps(activity.get("start_latlng")),
                "end_latlng": json.dumps(activity.get("end_latlng")),
                "average_speed": activity.get("average_speed"),
                "max_speed": activity.get("max_speed"),
                "average_cadence": activity.get("average_cadence"),
                "average_watts": activity.get("average_watts"),
                "max_watts": activity.get("max_watts"),
                "weighted_average_watts": activity.get("weighted_average_watts"),
                "device_watts": activity.get("device_watts"),
                "kilojoules": activity.get("kilojoules"),
                "has_heartrate": activity.get("has_heartrate"),
                "average_heartrate": activity.get("average_heartrate"),
                "max_heartrate": activity.get("max_heartrate"),
                "heartrate_opt_out": activity.get("heartrate_opt_out"),
                "display_hide_heartrate_option": activity.get("display_hide_heartrate_option"),
                "elev_high": activity.get("elev_high"),
                "elev_low": activity.get("elev_low"),
                "upload_id": activity.get("upload_id"),
                "upload_id_str": activity.get("upload_id_str"),
                "external_id": activity.get("external_id"),
                "from_accepted_tag": activity.get("from_accepted_tag"),
                "pr_count": activity.get("pr_count"),
                "total_photo_count": activity.get("total_photo_count"),
                "has_kudoed": activity.get("has_kudoed"),
                "import_date": datetime.now().isoformat()
            })
        except sqlite3.IntegrityError:
            print(f"Skipping duplicate activity with id {activity['id']}")

    conn.commit()
    conn.close()


#### Activities Batch Write

In [8]:
def insert_activities_batch(activity_list):
    """Efficiently insert multiple activity records into the database."""
    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()

    data = []
    for activity in activity_list:
        data.append({
            "id": activity["id"],
            "resource_state": activity.get("resource_state"),
            "athlete_id": activity.get("athlete", {}).get("id"),
            "athlete_resource_state": activity.get("athlete", {}).get("resource_state"),
            "name": activity.get("name"),
            "distance": activity.get("distance"),
            "moving_time": activity.get("moving_time"),
            "elapsed_time": activity.get("elapsed_time"),
            "total_elevation_gain": activity.get("total_elevation_gain"),
            "type": activity.get("type"),
            "sport_type": activity.get("sport_type"),
            "workout_type": activity.get("workout_type"),
            "start_date": activity.get("start_date"),
            "start_date_local": activity.get("start_date_local"),
            "timezone": activity.get("timezone"),
            "utc_offset": activity.get("utc_offset"),
            "location_city": activity.get("location_city"),
            "location_state": activity.get("location_state"),
            "location_country": activity.get("location_country"),
            "achievement_count": activity.get("achievement_count"),
            "kudos_count": activity.get("kudos_count"),
            "comment_count": activity.get("comment_count"),
            "athlete_count": activity.get("athlete_count"),
            "photo_count": activity.get("photo_count"),
            "map_id": activity.get("map", {}).get("id"),
            "map_summary_polyline": activity.get("map", {}).get("summary_polyline"),
            "map_resource_state": activity.get("map", {}).get("resource_state"),
            "trainer": activity.get("trainer"),
            "commute": activity.get("commute"),
            "manual": activity.get("manual"),
            "private": activity.get("private"),
            "visibility": activity.get("visibility"),
            "flagged": activity.get("flagged"),
            "gear_id": activity.get("gear_id"),
            "start_latlng": json.dumps(activity.get("start_latlng")),
            "end_latlng": json.dumps(activity.get("end_latlng")),
            "average_speed": activity.get("average_speed"),
            "max_speed": activity.get("max_speed"),
            "average_cadence": activity.get("average_cadence"),
            "average_watts": activity.get("average_watts"),
            "max_watts": activity.get("max_watts"),
            "weighted_average_watts": activity.get("weighted_average_watts"),
            "device_watts": activity.get("device_watts"),
            "kilojoules": activity.get("kilojoules"),
            "has_heartrate": activity.get("has_heartrate"),
            "average_heartrate": activity.get("average_heartrate"),
            "max_heartrate": activity.get("max_heartrate"),
            "heartrate_opt_out": activity.get("heartrate_opt_out"),
            "display_hide_heartrate_option": activity.get("display_hide_heartrate_option"),
            "elev_high": activity.get("elev_high"),
            "elev_low": activity.get("elev_low"),
            "upload_id": activity.get("upload_id"),
            "upload_id_str": activity.get("upload_id_str"),
            "external_id": activity.get("external_id"),
            "from_accepted_tag": activity.get("from_accepted_tag"),
            "pr_count": activity.get("pr_count"),
            "total_photo_count": activity.get("total_photo_count"),
            "has_kudoed": activity.get("has_kudoed"),
            "import_date": datetime.now().isoformat()
        })

    try:
        c.executemany('''
        INSERT OR IGNORE INTO activities VALUES (
            :id, :resource_state, :athlete_id, :athlete_resource_state,
            :name, :distance, :moving_time, :elapsed_time, :total_elevation_gain,
            :type, :sport_type, :workout_type, :start_date, :start_date_local,
            :timezone, :utc_offset, :location_city, :location_state, :location_country,
            :achievement_count, :kudos_count, :comment_count, :athlete_count, :photo_count,
            :map_id, :map_summary_polyline, :map_resource_state,
            :trainer, :commute, :manual, :private, :visibility, :flagged, :gear_id,
            :start_latlng, :end_latlng,
            :average_speed, :max_speed, :average_cadence, :average_watts,
            :max_watts, :weighted_average_watts, :device_watts, :kilojoules,
            :has_heartrate, :average_heartrate, :max_heartrate,
            :heartrate_opt_out, :display_hide_heartrate_option,
            :elev_high, :elev_low,
            :upload_id, :upload_id_str, :external_id, :from_accepted_tag,
            :pr_count, :total_photo_count, :has_kudoed, :import_date
        )
        ''', data)
    except sqlite3.Error as e:
        print("Error inserting batch:", e)

    conn.commit()
    conn.close()


#### Function to Rebuild single activity from flattened version in db

In [9]:
def get_activity_by_id(activity_id):
    """Retrieve a single activity and reconstruct its nested format."""
    conn = sqlite3.connect(DB_PATH)
    conn.row_factory = sqlite3.Row
    c = conn.cursor()

    c.execute("SELECT * FROM activities WHERE id = ?", (activity_id,))
    row = c.fetchone()
    conn.close()

    if row is None:
        return None

    return {
        "id": row["id"],
        "resource_state": row["resource_state"],
        "athlete": {
            "id": row["athlete_id"],
            "resource_state": row["athlete_resource_state"]
        },
        "name": row["name"],
        "distance": row["distance"],
        "moving_time": row["moving_time"],
        "elapsed_time": row["elapsed_time"],
        "total_elevation_gain": row["total_elevation_gain"],
        "type": row["type"],
        "sport_type": row["sport_type"],
        "workout_type": row["workout_type"],
        "start_date": row["start_date"],
        "start_date_local": row["start_date_local"],
        "timezone": row["timezone"],
        "utc_offset": row["utc_offset"],
        "location_city": row["location_city"],
        "location_state": row["location_state"],
        "location_country": row["location_country"],
        "achievement_count": row["achievement_count"],
        "kudos_count": row["kudos_count"],
        "comment_count": row["comment_count"],
        "athlete_count": row["athlete_count"],
        "photo_count": row["photo_count"],
        "map": {
            "id": row["map_id"],
            "summary_polyline": row["map_summary_polyline"],
            "resource_state": row["map_resource_state"]
        },
        "trainer": bool(row["trainer"]),
        "commute": bool(row["commute"]),
        "manual": bool(row["manual"]),
        "private": bool(row["private"]),
        "visibility": row["visibility"],
        "flagged": bool(row["flagged"]),
        "gear_id": row["gear_id"],
        "start_latlng": json.loads(row["start_latlng"]),
        "end_latlng": json.loads(row["end_latlng"]),
        "average_speed": row["average_speed"],
        "max_speed": row["max_speed"],
        "average_cadence": row["average_cadence"],
        "average_watts": row["average_watts"],
        "max_watts": row["max_watts"],
        "weighted_average_watts": row["weighted_average_watts"],
        "device_watts": bool(row["device_watts"]),
        "kilojoules": row["kilojoules"],
        "has_heartrate": bool(row["has_heartrate"]),
        "average_heartrate": row["average_heartrate"],
        "max_heartrate": row["max_heartrate"],
        "heartrate_opt_out": bool(row["heartrate_opt_out"]),
        "display_hide_heartrate_option": bool(row["display_hide_heartrate_option"]),
        "elev_high": row["elev_high"],
        "elev_low": row["elev_low"],
        "upload_id": row["upload_id"],
        "upload_id_str": row["upload_id_str"],
        "external_id": row["external_id"],
        "from_accepted_tag": bool(row["from_accepted_tag"]),
        "pr_count": row["pr_count"],
        "total_photo_count": row["total_photo_count"],
        "has_kudoed": bool(row["has_kudoed"])
    }


#### Simple Query to get records loaded during current day

In [10]:
def count_activities_imported_today(db_path):
    """Count how many activities were imported today based on the full ISO 8601 import_date timestamp."""
    conn = sqlite3.connect(db_path)
    c = conn.cursor()

    today_str = date.today().isoformat()  # 'YYYY-MM-DD'

    # Use substr to extract the date portion (first 10 characters)
    c.execute("""
        SELECT COUNT(*) 
        FROM activities 
        WHERE substr(import_date, 1, 10) = ?
    """, (today_str,))
    
    count = c.fetchone()[0]
    conn.close()
    return count


#### Simple Query to Latest record

In [11]:
def latest_activity_imported(db_path):
    """Count how many activities were imported today based on the full ISO 8601 import_date timestamp."""
    conn = sqlite3.connect(db_path)
    c = conn.cursor()

    today_str = date.today().isoformat()  # 'YYYY-MM-DD'

    # Use substr to extract the date portion (first 10 characters)
    c.execute("""
        SELECT * 
        FROM activities 
        WHERE import_date = 
        (select max(import_date) from activities)
    """)
    
    record = c.fetchone()
    conn.close()
    return record


In [62]:
### Simple Query to get Gear Id from Latest Record

In [95]:
def get_all_gear_ids(db_path):
    """Count how many activities were imported today based on the full ISO 8601 import_date timestamp."""
    conn = sqlite3.connect(db_path)
    c = conn.cursor()

    today_str = date.today().isoformat()  # 'YYYY-MM-DD'

    # Use substr to extract the date portion (first 10 characters)
    c.execute("""
        SELECT distinct gear_id 
        FROM activities 
    """)
    
    record = c.fetchone()
    conn.close()
    return record

#### Simple Query to Get Specific record

In [12]:
def activity_data_all(db_path, activity_id):
    """Count how many activities were imported today based on the full ISO 8601 import_date timestamp."""
    conn = sqlite3.connect(db_path)
    c = conn.cursor()

    # Run query and load into DataFrame (with column headers)
    df = pd.read_sql_query(
        "SELECT * FROM activities WHERE id = ?",
        conn,
        params=(activity_id,)
    )
    return df

#### Simple Query to get all Activity IDs

In [13]:
def get_all_activity_ids(db_path):
    """Count how many activities were imported today based on the full ISO 8601 import_date timestamp."""
    conn = sqlite3.connect(db_path)
    c = conn.cursor()

    today_str = date.today().isoformat()  # 'YYYY-MM-DD'

    # Use substr to extract the date portion (first 10 characters)
    c.execute("""
        SELECT id
        FROM activities 
        order by start_date desc
              """)
    
    count = c.fetchall()
    conn.close()
    return count

#### Simple Query to get all Activity IDs with HR and presumably Streams data

In [14]:
def get_all_activity_ids_with_HR(db_path):
    """Get all activity ids that have HR data, which hopefully will help us avoid a 404 call on the streams data.
    My thinking is that if no HR data, we probably have no streams data at all.
    """
    conn = sqlite3.connect(db_path)
    c = conn.cursor()

    today_str = date.today().isoformat()  # 'YYYY-MM-DD'

    # Use substr to extract the date portion (first 10 characters)
    c.execute("""
        SELECT id
        FROM activities 
        WHERE has_heartrate = 1
        order by start_date desc
              """)
    
    count = c.fetchall()
    conn.close()
    return count

## Database Streams Code

#### Initialize DB  

In [15]:
def initialize_streams_db():
    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()

    c.execute("""
    CREATE TABLE IF NOT EXISTS streams (
        activity_id INTEGER PRIMARY KEY,
        
        time_data TEXT,
        time_series_type TEXT,
        time_original_size INTEGER,
        time_resolution TEXT,

        distance_data TEXT,
        distance_series_type TEXT,
        distance_original_size INTEGER,
        distance_resolution TEXT,

        latlng_data TEXT,
        latlng_series_type TEXT,
        latlng_original_size INTEGER,
        latlng_resolution TEXT,

        altitude_data TEXT,
        altitude_series_type TEXT,
        altitude_original_size INTEGER,
        altitude_resolution TEXT,

        velocity_smooth_data TEXT,
        velocity_smooth_series_type TEXT,
        velocity_smooth_original_size INTEGER,
        velocity_smooth_resolution TEXT,

        heartrate_data TEXT,
        heartrate_series_type TEXT,
        heartrate_original_size INTEGER,
        heartrate_resolution TEXT,

        cadence_data TEXT,
        cadence_series_type TEXT,
        cadence_original_size INTEGER,
        cadence_resolution TEXT,

        watts_data TEXT,
        watts_series_type TEXT,
        watts_original_size INTEGER,
        watts_resolution TEXT,

        moving_data TEXT,
        moving_series_type TEXT,
        moving_original_size INTEGER,
        moving_resolution TEXT,

        grade_smooth_data TEXT,
        grade_smooth_series_type TEXT,
        grade_smooth_original_size INTEGER,
        grade_smooth_resolution TEXT
    )
    """)

    conn.commit()
    conn.close()

#### Insert Stream Data for Single Activity ID

In [16]:
def insert_stream_data(activity_id, stream_dict):
    """
    Inserts or replaces a row in the streams table for a given activity_id.
    stream_dict should have keys like 'time', 'distance', etc., with each value a dict containing:
    {
        'data': [...],
        'series_type': '...',
        'original_size': ...,
        'resolution': '...'
    }
    """
    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()

    # Create the column mappings dynamically
    base_columns = []
    placeholders = []
    values = []

    # Always include activity_id
    base_columns.append("activity_id")
    placeholders.append("?")
    values.append(activity_id)

    for key, val in stream_dict.items():
        if not isinstance(val, dict):
            continue  # skip malformed

        base_columns.extend([
            f"{key}_data",
            f"{key}_series_type",
            f"{key}_original_size",
            f"{key}_resolution"
        ])
        placeholders.extend(["?"] * 4)

        values.extend([
            json.dumps(val.get("data")),
            val.get("series_type"),
            val.get("original_size"),
            val.get("resolution")
        ])

    sql = f"""
        INSERT INTO streams ({', '.join(base_columns)})
        VALUES ({', '.join(placeholders)})
    """
    

    try:
        c.execute(sql, values)
        conn.commit()
        conn.close()
    except sqlite3.IntegrityError:
        print(f"Activity {activity_id} already exists in the 'streams' table. Skipping insert.")

#### Simple Query to Get all activity IDs from Streams

In [17]:
def get_all_activity_ids_streams(db_path):
    """Count how many activities were imported today based on the full ISO 8601 import_date timestamp."""
    conn = sqlite3.connect(db_path)
    c = conn.cursor()

    today_str = date.today().isoformat()  # 'YYYY-MM-DD'

    # Use substr to extract the date portion (first 10 characters)
    c.execute("""
        SELECT activity_id
        FROM streams 
              """)
    
    count = c.fetchall()
    conn.close()
    return count

In [18]:
#### Simple Query to Get all Streams for latest activity

In [19]:
def single_stream_example(db_path):
    """Count how many activities were imported today based on the full ISO 8601 import_date timestamp."""
    conn = sqlite3.connect(db_path)
    
    # Run query and load into DataFrame (with column headers)
    df = pd.read_sql_query(
        """
        SELECT * 
        FROM streams 
        ORDER BY activity_id desc
        limit 1
    """,
        conn
    )
    return df


In [20]:
test_df = single_stream_example(DB_PATH)
test_df 

,activity_id,time_data,time_series_type,time_original_size,time_resolution,distance_data,distance_series_type,distance_original_size,distance_resolution,latlng_data,latlng_series_type,latlng_original_size,latlng_resolution,altitude_data,altitude_series_type,altitude_original_size,altitude_resolution,velocity_smooth_data,velocity_smooth_series_type,velocity_smooth_original_size,velocity_smooth_resolution,heartrate_data,heartrate_series_type,heartrate_original_size,heartrate_resolution,cadence_data,cadence_series_type,cadence_original_size,cadence_resolution,watts_data,watts_series_type,watts_original_size,watts_resolution,moving_data,moving_series_type,moving_original_size,moving_resolution,grade_smooth_data,grade_smooth_series_type,grade_smooth_original_size,grade_smooth_resolution
0,14393650080,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",distance,3049,high,"[0.0, 0.0, 0.0, 2.0, 4.0, 6.0, 8.0, 10.0, 12.0...",distance,3049,high,"[[44.949551, -93.324906], [44.949551, -93.3249...",distance,3049,high,"[278.0, 278.0, 278.0, 278.0, 278.0, 278.0, 278...",distance,3049,high,"[0.0, 0.0, 0.0, 0.667, 1.0, 1.2, 1.6, 2.0, 2.0...",distance,3049,high,"[61, 61, 61, 61, 61, 61, 62, 63, 63, 63, 63, 6...",distance,3049,high,"[86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 87, 8...",distance,3049,high,"[null, null, null, null, null, null, null, nul...",distance,3049,high,"[false, false, false, true, true, true, true, ...",distance,3049,high,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",distance,3049,high


In [21]:
len(test_df.time_data[0])

17184

In [22]:
len(test_df.heartrate_data[0])

15197

In [23]:
def get_streams_data(activity_id, db_path):
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    cur.execute("SELECT distance_data, heartrate_data, altitude_data FROM streams WHERE activity_id = ?", (activity_id,))
    row = cur.fetchone()
    conn.close()

    if row:
        try:
            distance = json.loads(row[0]) if row[0] else []
            heartrate = json.loads(row[1]) if row[1] else []
            altitude = json.loads(row[2]) if row[2] else []
            return distance, heartrate, altitude
        except Exception as e:
            print("Failed to load stream data:", e)
    return [], [], []

In [107]:
z = get_streams_data(14414666612, DB_PATH)

OperationalError: no such column: id

In [105]:
z

([], [], [])

## Database Gear Code

#### Initialize Gear Table  

In [74]:
def initialize_gear_db():
    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()

    c.execute("""
    CREATE TABLE IF NOT EXISTS gear (
        gear_id TEXT PRIMARY KEY,
        is_primary BOOLEAN,
        resource_state INTEGER,
        distance INTEGER,
        brand_name TEXT,
        model_name TEXT,      
        frame_type INTEGER,
        description TEXT
    )
    """)

    conn.commit()
    conn.close()

#### Insert Single Gear

In [93]:
import json

def insert_single_gear(gear):
    """Insert activity records, skipping those with duplicate 'id'."""
    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()
    try:
        c.execute('''
        INSERT OR REPLACE INTO gear VALUES (
            :gear_id, :is_primary, 
            :resource_state, :distance,
            :brand_name, :model_name, :frame_type, :description
                  )
        ''', {
            "gear_id": gear["id"],
            "is_primary": gear.get("primary"),
            "resource_state": gear.get("resource_state"),
            "distance": gear.get("distance"),
            "brand_name": gear.get("brand_name"),
            "model_name": gear.get("model_name"),
            "frame_type": gear.get("frame_type"),
            "description": gear.get("description"),
            "import_date": datetime.now().isoformat()
        })
    except sqlite3.IntegrityError:
        print(f"Skipping duplicate activity with id {gear['id']}")

    conn.commit()
    conn.close()

## Get Latest Activity and Associated Streams

In [14]:
try:
    activities = []
    streams = []
    page = 1
    per_page = 1
    while True:
        activities_page = get_activities(AUTH_TOKEN, page=page, per_page=per_page)
        if not activities_page:
            break
        activities.extend(activities_page)
        page += 1
        time.sleep(1)  # Respect rate limits, adjust if needed
        break

    # Process the activity data
    for activity in activities:
        print(f"Activity ID: {activity['id']}, Name: {activity['name']}, Type: {activity['type']}")

    keys = ["time", "distance", "latlng", "altitude", "heartrate", "cadence", "watts"]
    keys = "time, latlng"
    # Process stream data
    for a in activities:
        stream = get_streams(AUTH_TOKEN, keys, a['id'])
        if not stream:
            break
        streams.extend(stream)
        time.sleep(1)  # Respect rate limits, adjust if needed
        break

except requests.exceptions.HTTPError as e:
    print(f"HTTP error occurred: {e}")
except json.JSONDecodeError as e:
    print(f"JSON decode error occurred: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Activity ID: 14404286770, Name: Morning Run, Type: Run


## Bulk Historical Processing of Activities

In [ ]:
# starting point, build unix timestamps in 6 month batches

# time_0 = time.mktime(datetime.datetime(2019, 1, 1, 0, 0, 0).timetuple())
# time_1 = time.mktime(datetime.datetime(2019, 7, 1, 0, 0, 0).timetuple()) 
# time_2 = time.mktime(datetime.datetime(2020, 1, 1, 0, 0, 0).timetuple()) 
# time_3 = time.mktime(datetime.datetime(2020, 7, 1, 0, 0, 0).timetuple())
# time_4 = time.mktime(datetime.datetime(2021, 1, 1, 0, 0, 0).timetuple())
# time_5 = time.mktime(datetime.datetime(2021, 7, 1, 0, 0, 0).timetuple())
# time_6 = time.mktime(datetime.datetime(2022, 1, 1, 0, 0, 0).timetuple())
# time_7 = time.mktime(datetime.datetime(2022, 7, 1, 0, 0, 0).timetuple())
# time_8 = time.mktime(datetime.datetime(2023, 1, 1, 0, 0, 0).timetuple())
# time_9 = time.mktime(datetime.datetime(2023, 7, 1, 0, 0, 0).timetuple())
# time_10 = time.mktime(datetime.datetime(2024, 1, 1, 0, 0, 0).timetuple())
# time_11 = time.mktime(datetime.datetime(2024, 7, 1, 0, 0, 0).timetuple())
# time_12 = time.mktime(datetime.datetime(2025, 1, 1, 0, 0, 0).timetuple())
# time_13 = time.mktime(datetime.datetime(2025, 7, 1, 0, 0, 0).timetuple())

day_by_day_before = time.mktime(datetime(2025, 5, 10, 0, 0, 0).timetuple())
day_by_day_after = time.mktime(datetime(2025, 5, 9, 0, 0, 0).timetuple())

In [85]:
# ensure db table exists
initialize_database()

# Get the data
try:
    activities = []
    page = 1
    per_page = 30
    while True:
        activities_page = get_activities(AUTH_TOKEN, page=page, per_page=per_page, before = day_by_day_before, after = day_by_day_after) # currently done, latest after time was time_12
        if not activities_page:
            break
        print(len(activities_page)) # expect 30 each time unless final page
        activities.extend(activities_page)
        
        insert_activities_batch(activities_page) # attempt to bulk write to db
        time.sleep(1)  # Respect rate limits, adjust if needed

        num_today = count_activities_imported_today(DB_PATH) # count objects in db now
        print(f"{num_today} activities were imported so far.")

        page += 1
        time.sleep(1)  # Respect rate limits, adjust if needed
        if page > 50:
            break

except requests.exceptions.HTTPError as e:
    print(f"HTTP error occurred: {e}")
except json.JSONDecodeError as e:
    print(f"JSON decode error occurred: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

num_today = count_activities_imported_today(DB_PATH)
print(f"{num_today} activities were imported so far today")
print(f"{len(activities)} activities were imported from the API this round.")

2
1 activities were imported so far.
1 activities were imported so far today
2 activities were imported from the API this round.


## Bulk Historical Processing of Data Streams

In [58]:
# Initialize the DB
initialize_streams_db()

# get all stream activity IDs
stream_activity_ids = get_all_activity_ids_streams("strava_data.db")
stream_activity_ids = set(i[0] for i in stream_activity_ids)

# get all activity ID's in db sorted by activity date desc as set
sorted_activity_list = get_all_activity_ids_with_HR("strava_data.db")
sorted_activity_list = set(i[0] for i in sorted_activity_list)

# Take the 
valid_activity_ids = tuple(sorted_activity_list - stream_activity_ids)

keys = "time,distance,latlng,altitude,velocity_smooth,heartrate,cadence,watts,temp,moving,grade_smooth"

for i, activity_integer in enumerate(valid_activity_ids):
    
    try:
        stream = get_streams(AUTH_TOKEN, keys, activity_integer)
        if not stream:
            print('no stream')
            continue
    
    except requests.exceptions.HTTPError as e:
        print(f"HTTP error occurred: {e}")
        if "429" in str(e):
            print("RATE LIMITED!!!")
            break
        if "404" in str(e):
            print("Stream data unavailable for activity")
            continue
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    time.sleep(1)  # Respect rate limits, adjust if needed
    insert_stream_data(activity_integer, stream)
    time.sleep(1)  # Respect rate limits, adjust if needed
    if i > 98:
        break



In [59]:
num_today = count_activities_imported_today(DB_PATH)
print(f"{len(get_all_activity_ids_streams("strava_data.db"))} streams in the db.")
print(f"{len(get_all_activity_ids_with_HR("strava_data.db"))} valid activities with potential streams data exist in the db.")
print(f"{len(get_all_activity_ids_with_HR("strava_data.db")) - len(get_all_activity_ids_streams("strava_data.db"))} to go!")

2033 streams in the db.
2746 valid activities with potential streams data exist in the db.
713 to go!


## Gear Workspace

In [96]:
# Initialize the DB
initialize_gear_db()

shoe_id = get_all_gear_ids(DB_PATH)
shoe_id



('b5036222',)

In [109]:
shoe = get_gear(AUTH_TOKEN, "b6893678")

In [88]:
shoe

{'id': 'g15302697',
 'primary': False,
 'name': 'New Balance Vongo v5 🦇',
 'nickname': '🦇',
 'resource_state': 3,
 'retired': False,
 'distance': 1076704,
 'converted_distance': 669.0,
 'brand_name': 'New Balance',
 'model_name': 'Vongo v5',
 'description': None,
 'notification_distance': 450}

In [110]:
insert_single_gear(shoe)